<a href="https://colab.research.google.com/github/AxelADN/Multiagent_Systems_Simulation_Examples/blob/main/Hangman_Game_Agent_Messaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install agentpy owlready2

# The Hangman Game

In [ ]:
from owlready2 import *
import agentpy as ap
import string
import random

###Ontology

In [ ]:
onto = get_ontology("file:///content/VaccumOnto")

In [ ]:
with onto:

    class Hangman(Thing):
        pass

    class Player(Thing):
        pass

    class Host(Player):
        pass

    class Guesser(Player):
        pass

    class letters_guessed(FunctionalProperty,DataProperty):
        domain = [Hangman]
        range = [int]

    class letters_missing(FunctionalProperty,DataProperty):
        domain = [Hangman]
        range = [int]
        pass

    class progress(FunctionalProperty,DataProperty):
        domain = [Hangman]
        range = [float]

    class the_hangman(FunctionalProperty,ObjectProperty):
        domain = [Player]
        range = [Hangman]

    class is_communicating(FunctionalProperty,DataProperty):
        domain = [Player]
        range = [str]

    class played_word(FunctionalProperty,DataProperty):
        domain = [Hangman]
        range = [str]

    class left_letters(FunctionalProperty,DataProperty):
        domain = [Guesser]
        range = [str]


###Message Manager

In [ ]:
#MESSAGE CLASS
#Class to handle and build a message, based on KQML
class Message():

    performatives = ["request","inform"]
    parameters = ["content","sender","reply-with","in-reply-to"]

    def __init__(self,msg="",performative="",content="",sender="",query="q1",is_reply=True):
        """Constructor to build a new message"""
        self.empty = False
        self.request = False
        self.inform = False
        self.msg = msg

        #If we want to build a message from the paramters
        if msg == "":
            self.is_reply = is_reply
            self.query = query
            assert performative in Message.performatives , f"Performaive: {performative}"
            self.performative=performative
            self.content = content
            self.sender = sender

        #if we want to build a message from a string (a KQML message)
        else:
            self.decode()

        #Identify if its either Request or Inform performative
        if self.performative == "request":
            self.request = True
        elif self.performative == "inform":
            self.inform = True
        else:
            self.empty = True

    def decode(self):
        """Method to convert a string message (KQML format) to message parameters"""
        current = self.msg[1:-1]
        current = current.split("\n")
        self.performative = current[0]
        assert self.performative in Message.performatives , f"Performaive: {self.performative}"
        parameterList = current[1].split(":")[1:]
        parametersDict = {}
        for parameter in parameterList:
            pair = parameter.split(" ")
            parametersDict[pair[0]] = pair[1]
        if "in-reply-to" in parametersDict.keys():
            self.query = parametersDict["in-reply-to"]
            self.is_reply = True
        else:
            self.query = parametersDict["reply-with"]
            self.is_reply = False
        self.content = parametersDict["content"]
        self.sender = parametersDict["sender"]

    def __str__(self):
        """Method to convert message paramters to a string (KQML format)"""
        s = "("
        s+= self.performative + "\n"
        s+= ":sender " + self.sender
        s+= ":content "+self.content
        if self.is_reply:
            s+= ":in-reply-to " + self.query
        else:
            s+= ":reply-with " + self.query
        s+= ")"
        return s




###Agent Classes

In [ ]:
class GuesserPlayerAgent(ap.Agent):

    def see(self):
        #The see() function that grabs the message from the 'broadcast stream'
        p = self.model.broadcastMessage
        return p

    def brf(self,p):
        self.msg = Message(msg=p)
        print()
        print("Guesser Message:")
        print(self.msg)
        print()
        self.host.is_communicating = str(self.msg)

    def options(self):
        if self.msg.empty:
            self.D = []
            return
        #The following possible options are based from the content of the message
        #And if its performative is of type "inform"
        if self.msg.inform:
            if "word" in self.msg.content.split("-"):
                print(f"Guesser received word {self.msg.content.split('-')[1]}")
                leftLetters = eval(self.thisGuesser.left_letters)
                if len(leftLetters ) > 0:
                    letter = random.choice(leftLetters)
                    leftLetters.remove(letter)
                    self.thisGuesser.left_letters = str(leftLetters)
                    self.D = [letter]
                    return
            if "start" in self.msg.content.split("-"):
                leftLetters = eval(self.thisGuesser.left_letters)
                letter = random.choice(leftLetters)
                leftLetters.remove(letter)
                self.thisGuesser.left_letters = str(leftLetters)
                self.D = [letter]
                return
            if "finished" in self.msg.content.split("-"):
                self.D = []
        self.D = []


    def filter(self):
        if len(self.D) <=0:
            self.I = "_"
            return
        self.I = self.D[0]


    def plan(self):
        self.thePlan = self.I


    def BDI(self):
        self.brf(self.see())
        self.options()
        self.filter()
        self.plan()

    def execute(self):
        #The action of the agent is to send a message (request) to the broadcast stream
        self.model.broadcastMessage = str(Message(performative="request", content="letter-"+self.I,sender=self.thisGuesser.name))

    def setup(self):
        self.firstStep = True
        self.finished = False
        self.thePlan = {}

    def step(self):
        if self.firstStep:
            self.thisGuesser = Guesser(left_letters = str(list(string.ascii_lowercase)))
            self.host = Host()
            self.firstStep = False

        self.BDI()
        self.execute()

    def update(self):
        pass

    def end(self):
        pass

In [ ]:
class HostPlayerAgent(ap.Agent):


    def see(self):
        #The see() function that grabs the message from the 'broadcast stream'
        p = self.model.broadcastMessage
        return p

    def brf(self,p):
        self.msg = Message(msg=p)
        print()
        print("Host Message:")
        print(self.msg)
        self.guesser.is_communicating = str(self.msg)
        self.thisHost.the_hangman.letters_guessed = len(self.structWord) - self.structWord.count("_")
        self.thisHost.the_hangman.letters_missing = self.structWord.count("_")
        #self.thisHost.the_hangman.progress = 1/self.thisHost.the_hangman.letters_missing

    def options(self):
        if self.msg.empty:
            self.D = []
            return
        #The following possible options are based from the content of the message
        #And if its performative is of type "request"
        if self.msg.request:
            if "letter" in self.msg.content.split("-"):
                self.D = [self.msg.content.split("-")[1]]
                return
        self.D = []


    def filter(self):
        if len(self.D) <=0:
            self.I = "_"
            return
        self.I = self.D[0]


    def plan(self):
        theWord = self.thisHost.the_hangman.played_word
        indexes = [i for i,s in enumerate(list(theWord)) if s == self.I]
        self.thePlan = {k:self.I for k in indexes}


    def BDI(self):
        self.brf(self.see())
        self.options()
        self.filter()
        self.plan()

    def execute(self):
        #The action of the agent is to modify the target word for the game and
        #send a message (inform) to the broadcast stream about the status of the word
        for i in self.thePlan.keys():
            self.structWord[i] = self.thePlan[i]
        print(f'Chosen letter: {self.I} \| The Word: {self.structWord}')
        s = ""
        for c in self.structWord:
            s += c
        if not "_" in self.structWord:
            self.model.broadcastMessage = str(Message(performative="inform", content="finish-",sender=self.thisHost.name))
        self.model.broadcastMessage = str(Message(performative="inform", content="word-"+s,sender=self.thisHost.name))

    def setup(self):
        self.firstStep = True
        self.structWord = ["_"]
        self.thePlan = {}

    def step(self):

        if self.firstStep:
            #Late-initialize the Beliefs and other parameters
            myHangman = Hangman()
            myHangman.progress = 0.0
            myHangman.letters_guessed = 0
            myHangman.letters_missing = 0
            #Choose random word for the game
            myHangman.played_word = random.choice(["dog","alphabet","color","computer","desoxiribonuclei"])
            self.thisHost = Host(the_hangman=myHangman)
            self.guesser = Guesser()
            self.model.broadcastMessage = str(Message(performative="inform",content="start-",sender=self.thisHost.name,is_reply=False))
            self.structWord = ["_" for _ in range(len(myHangman.played_word))]
            self.firstStep = False

        self.BDI()
        self.execute()

    def update(self):
        #If there are no letters left to guess then stop de simulation
        if not "_" in self.structWord:
            #pass
            self.model.stop()
        #I think I'm missing something because I don't see this working...
        pass

    def end(self):
        pass


###The Model

In [ ]:
class HangmanModel(ap.Model):
    def setup(self):
        #The 'Broadcast Stream' as an attribute.
        self.broadcastMessage = ""

        self.hosts = ap.AgentList(self, 1, HostPlayerAgent)
        self.guessers = ap.AgentList(self, 1, GuesserPlayerAgent)
        pass

    def step(self):
        self.hosts.step()
        self.guessers.step()
        pass

    def update(self):
        self.hosts.update()
        self.guessers.update()
        pass

    def end(self):
        pass

###Execution

In [ ]:
parameters = {
    "steps" : 100
}

model = HangmanModel(parameters)
model.run()


Host Message:
(inform
:sender host1:content start-:reply-with q1)
Chosen letter: _ \| The Word: ['_', '_', '_']

Guesser Message:
(inform
:sender host1:content word-___:in-reply-to q1)

Guesser received word ___
Completed: 1 steps
Host Message:
(request
:sender guesser2:content letter-n:in-reply-to q1)
Chosen letter: n \| The Word: ['_', '_', '_']

Guesser Message:
(inform
:sender host1:content word-___:in-reply-to q1)

Guesser received word ___
Completed: 2 steps
Host Message:
(request
:sender guesser2:content letter-s:in-reply-to q1)
Chosen letter: s \| The Word: ['_', '_', '_']

Guesser Message:
(inform
:sender host1:content word-___:in-reply-to q1)

Guesser received word ___
Completed: 3 steps
Host Message:
(request
:sender guesser2:content letter-i:in-reply-to q1)
Chosen letter: i \| The Word: ['_', '_', '_']

Guesser Message:
(inform
:sender host1:content word-___:in-reply-to q1)

Guesser received word ___
Completed: 4 steps
Host Message:
(request
:sender guesser2:content letter

DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 1 key
'reporters': DataFrame with 1 variable and 1 row
}